In [32]:
from __future__ import print_function
import sys
import pygame 
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

In [42]:
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

def load_music_utils():
    
    chords, abstract_grammars = get_musical_data('data/original_metheny.mid')
    corpus, tones, tones_indices, indices_tones = get_corpus_data(abstract_grammars)
    N_tones = len(set(corpus))
    X, Y, N_tones = data_processing(corpus, tones_indices, 60, 30)   
    
    return (X, Y, N_tones, indices_tones)


def generate_music(inference_model, corpus = corpus, abstract_grammars = abstract_grammars, tones = tones, tones_indices = tones_indices, indices_tones = indices_tones, T_y = 10, max_tries = 1000, diversity = 0.5):
    # set up audio stream
    out_stream = stream.Stream()
    
    # Initialize chord variables
    curr_offset = 0.0                                     # variable used to write sounds to the Stream.
    num_chords = int(len(chords) / 3)                     # number of different set of chords
    
    print("Predicting new values for different set of chords.")
    # Loop over all 18 set of chords. At each iteration generate a sequence of tones
    # and use the current chords to convert it into actual sounds 
    for i in range(1, num_chords):
        
        # Retrieve current chord from stream
        curr_chords = stream.Voice()
        
        # Loop over the chords of the current set of chords
        for j in chords[i]:
            # Add chord to the current chords with the adequate offset, no need to understand this
            curr_chords.insert((j.offset % 4), j)
        
        # Generate a sequence of tones using the model
        _, indices = predict_and_sample(inference_model)
        indices = list(indices.squeeze())
        pred = [indices_tones[p] for p in indices]
        
        predicted_tones = 'C,0.25 '
        for k in range(len(pred) - 1):
            predicted_tones += pred[k] + ' ' 
        
        predicted_tones +=  pred[-1]
                
        #### POST PROCESSING OF THE PREDICTED TONES ####
        # We will consider "A" and "X" as "C" tones. It is a common choice.
        predicted_tones = predicted_tones.replace(' A',' C').replace(' X',' C')

        # Pruning #1: smoothing measure
        predicted_tones = prune_grammar(predicted_tones)
        
        # Use predicted tones and current chords to generate sounds
        sounds = unparse_grammar(predicted_tones, curr_chords)

        # Pruning #2: removing repeated and too close together sounds
        sounds = prune_notes(sounds)

        # Quality assurance: clean up sounds
        sounds = clean_up_notes(sounds)

        # Print number of tones/notes in sounds
        print('Generated %s sounds using the predicted values for the set of chords ("%s") and after pruning' % (len([k for k in sounds if isinstance(k, note.Note)]), i))
        
        # Insert sounds into the output stream
        for m in sounds:
            out_stream.insert(curr_offset + m.offset, m)
        for mc in curr_chords:
            out_stream.insert(curr_offset + mc.offset, mc)

        curr_offset += 4.0
        
    # Initialize tempo of the output stream with 130 bit per minute
    out_stream.insert(0.0, tempo.MetronomeMark(number=130))

    # Save audio stream to fine
    mf = midi.translate.streamToMidiFile(out_stream)
    mf.open("output/my_music.midi", 'wb')
    mf.write()
    print("Your generated music is saved in output/my_music.midi")
    mf.close()
    
    # Play the final stream through output (see 'play' lambda function above)
    # play = lambda x: midi.realtime.StreamPlayer(x).play()
    # play(out_stream)
    
    return out_stream


def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, c_initializer = c_initializer):

    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    indices = np.argmax(pred, axis = -1)
    results = to_categorical(indices, num_classes=78)
    
    return results, indices

def play_music(music_file):

    try:
        pygame.mixer.music.load(music_file)
        
    except pygame.error:
        print ("Couldn't play %s! (%s)" % (music_file, pygame.get_error()))
        return
        
    pygame.mixer.music.play()

<music21.stream.Part 0x14f6d70ab38>


In [24]:
def djmodel(Tx, n_a, n_values):

    X = Input(shape=(Tx, n_values))

    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    outputs = []

    for t in range(Tx):

        x =  Lambda(lambda x: X[:, t, :])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        out = densor(a)
        outputs.append(out)

    model = Model(inputs=[X,a0,c0],outputs=outputs)

    return model


def music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):

    x0 = Input(shape=(1, n_values))

    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0
    outputs = []

    for t in range(Ty):
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        out = densor(a)
        outputs.append(out)
        x =  Lambda(one_hot)(out)

    inference_model = Model(inputs=[x0,a0,c0],outputs=outputs)

    return inference_model

def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, c_initializer = c_initializer):

    pred = inference_model.predict([x_initializer,a_initializer,c_initializer])
    indices = np.argmax(pred, axis = -1)
    results = to_categorical(indices,num_classes=78)

    return results, indices

In [43]:
play_music('data/original_metheny.mid')

In [5]:
X, Y, n_values, indices_values = load_music_utils()

n_a = 64 
m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))
reshapor = Reshape((1, 78))                       
LSTM_cell = LSTM(n_a, return_state = True)         
densor = Dense(n_values, activation='softmax') 

In [6]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
60/60 [==============================] - 78s 1s/step - loss: 125.8816 - dense_1_loss_1: 4.3548 - dense_1_loss_2: 4.3491 - dense_1_loss_3: 4.3461 - dense_1_loss_4: 4.3422 - dense_1_loss_5: 4.3511 - dense_1_loss_6: 4.3450 - dense_1_loss_7: 4.3463 - dense_1_loss_8: 4.3471 - dense_1_loss_9: 4.3420 - dense_1_loss_10: 4.3381 - dense_1_loss_11: 4.3384 - dense_1_loss_12: 4.3400 - dense_1_loss_13: 4.3436 - dense_1_loss_14: 4.3283 - dense_1_loss_15: 4.3370 - dense_1_loss_16: 4.3307 - dense_1_loss_17: 4.3474 - dense_1_loss_18: 4.3408 - dense_1_loss_19: 4.3277 - dense_1_loss_20: 4.3414 - dense_1_loss_21: 4.3388 - dense_1_loss_22: 4.3402 - dense_1_loss_23: 4.3412 - dense_1_loss_24: 4.3379 - dense_1_loss_25: 4.3473 - dense_1_loss_26: 4.3328 - dense_1_loss_27: 4.3360 - dense_1_loss_28: 4.3355 - dense_1_loss_29: 4.3349 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0000e+00 - dense_1_acc_2: 0.0500 - dense_1_acc_3: 0.0333 - dense_1_acc_4: 0.0667 - dense_1_acc_5: 0.0333 - dense_1_acc_6: 0.

60/60 [==============================] - 1s 13ms/step - loss: 109.4408 - dense_1_loss_1: 4.2582 - dense_1_loss_2: 4.1334 - dense_1_loss_3: 3.9933 - dense_1_loss_4: 3.9531 - dense_1_loss_5: 3.8556 - dense_1_loss_6: 3.8834 - dense_1_loss_7: 3.8452 - dense_1_loss_8: 3.6456 - dense_1_loss_9: 3.7051 - dense_1_loss_10: 3.5480 - dense_1_loss_11: 3.6605 - dense_1_loss_12: 3.9240 - dense_1_loss_13: 3.6203 - dense_1_loss_14: 3.6190 - dense_1_loss_15: 3.7064 - dense_1_loss_16: 3.6929 - dense_1_loss_17: 3.7960 - dense_1_loss_18: 3.7263 - dense_1_loss_19: 3.6760 - dense_1_loss_20: 3.8422 - dense_1_loss_21: 3.8317 - dense_1_loss_22: 3.7187 - dense_1_loss_23: 3.6603 - dense_1_loss_24: 3.6400 - dense_1_loss_25: 3.8708 - dense_1_loss_26: 3.5398 - dense_1_loss_27: 3.6107 - dense_1_loss_28: 3.6340 - dense_1_loss_29: 3.8505 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.1000 - dense_1_acc_2: 0.1000 - dense_1_acc_3: 0.1833 - dense_1_acc_4: 0.1833 - dense_1_acc_5: 0.2167 - dense_1_acc_6: 0.0833 - dense_1_

60/60 [==============================] - 1s 14ms/step - loss: 94.5315 - dense_1_loss_1: 4.1964 - dense_1_loss_2: 3.9628 - dense_1_loss_3: 3.6480 - dense_1_loss_4: 3.6017 - dense_1_loss_5: 3.3917 - dense_1_loss_6: 3.4580 - dense_1_loss_7: 3.3928 - dense_1_loss_8: 3.1487 - dense_1_loss_9: 3.1589 - dense_1_loss_10: 2.9370 - dense_1_loss_11: 3.1116 - dense_1_loss_12: 3.2619 - dense_1_loss_13: 3.0429 - dense_1_loss_14: 3.0115 - dense_1_loss_15: 3.0661 - dense_1_loss_16: 3.1101 - dense_1_loss_17: 2.9979 - dense_1_loss_18: 3.1898 - dense_1_loss_19: 2.9525 - dense_1_loss_20: 3.1684 - dense_1_loss_21: 3.2837 - dense_1_loss_22: 3.1215 - dense_1_loss_23: 3.1422 - dense_1_loss_24: 3.1162 - dense_1_loss_25: 3.4533 - dense_1_loss_26: 2.9425 - dense_1_loss_27: 3.1683 - dense_1_loss_28: 3.1659 - dense_1_loss_29: 3.3289 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.1000 - dense_1_acc_2: 0.1500 - dense_1_acc_3: 0.2333 - dense_1_acc_4: 0.1833 - dense_1_acc_5: 0.2167 - dense_1_acc_6: 0.1167 - dense_1_a

60/60 [==============================] - 1s 14ms/step - loss: 75.0183 - dense_1_loss_1: 4.1542 - dense_1_loss_2: 3.7764 - dense_1_loss_3: 3.3018 - dense_1_loss_4: 3.1570 - dense_1_loss_5: 2.8426 - dense_1_loss_6: 2.8866 - dense_1_loss_7: 2.7743 - dense_1_loss_8: 2.5169 - dense_1_loss_9: 2.4632 - dense_1_loss_10: 2.3977 - dense_1_loss_11: 2.4594 - dense_1_loss_12: 2.4344 - dense_1_loss_13: 2.2873 - dense_1_loss_14: 2.2601 - dense_1_loss_15: 2.3397 - dense_1_loss_16: 2.4609 - dense_1_loss_17: 2.2638 - dense_1_loss_18: 2.4020 - dense_1_loss_19: 2.2215 - dense_1_loss_20: 2.3930 - dense_1_loss_21: 2.4098 - dense_1_loss_22: 2.2570 - dense_1_loss_23: 2.3995 - dense_1_loss_24: 2.3913 - dense_1_loss_25: 2.6908 - dense_1_loss_26: 2.0475 - dense_1_loss_27: 2.3416 - dense_1_loss_28: 2.2715 - dense_1_loss_29: 2.4162 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.1000 - dense_1_acc_2: 0.1833 - dense_1_acc_3: 0.2833 - dense_1_acc_4: 0.2500 - dense_1_acc_5: 0.3000 - dense_1_acc_6: 0.2167 - dense_1_a

60/60 [==============================] - 1s 15ms/step - loss: 56.7054 - dense_1_loss_1: 4.1162 - dense_1_loss_2: 3.5385 - dense_1_loss_3: 2.8508 - dense_1_loss_4: 2.5937 - dense_1_loss_5: 2.2367 - dense_1_loss_6: 2.1407 - dense_1_loss_7: 2.0113 - dense_1_loss_8: 1.8280 - dense_1_loss_9: 1.7937 - dense_1_loss_10: 1.6900 - dense_1_loss_11: 1.7904 - dense_1_loss_12: 1.6887 - dense_1_loss_13: 1.5164 - dense_1_loss_14: 1.5854 - dense_1_loss_15: 1.7039 - dense_1_loss_16: 1.8010 - dense_1_loss_17: 1.5877 - dense_1_loss_18: 1.6581 - dense_1_loss_19: 1.6004 - dense_1_loss_20: 1.6484 - dense_1_loss_21: 1.7124 - dense_1_loss_22: 1.6619 - dense_1_loss_23: 1.6557 - dense_1_loss_24: 1.7060 - dense_1_loss_25: 1.9257 - dense_1_loss_26: 1.6215 - dense_1_loss_27: 1.7251 - dense_1_loss_28: 1.6149 - dense_1_loss_29: 1.7022 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.2167 - dense_1_acc_3: 0.3333 - dense_1_acc_4: 0.3167 - dense_1_acc_5: 0.4333 - dense_1_acc_6: 0.4167 - dense_1_a

...

60/60 [==============================] - 1s 13ms/step - loss: 6.9040 - dense_1_loss_1: 3.8054 - dense_1_loss_2: 1.4493 - dense_1_loss_3: 0.5013 - dense_1_loss_4: 0.1385 - dense_1_loss_5: 0.0862 - dense_1_loss_6: 0.0647 - dense_1_loss_7: 0.0548 - dense_1_loss_8: 0.0429 - dense_1_loss_9: 0.0421 - dense_1_loss_10: 0.0360 - dense_1_loss_11: 0.0419 - dense_1_loss_12: 0.0373 - dense_1_loss_13: 0.0328 - dense_1_loss_14: 0.0331 - dense_1_loss_15: 0.0336 - dense_1_loss_16: 0.0335 - dense_1_loss_17: 0.0331 - dense_1_loss_18: 0.0356 - dense_1_loss_19: 0.0350 - dense_1_loss_20: 0.0366 - dense_1_loss_21: 0.0362 - dense_1_loss_22: 0.0341 - dense_1_loss_23: 0.0344 - dense_1_loss_24: 0.0332 - dense_1_loss_25: 0.0351 - dense_1_loss_26: 0.0357 - dense_1_loss_27: 0.0408 - dense_1_loss_28: 0.0388 - dense_1_loss_29: 0.0421 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.5833 - dense_1_acc_3: 0.8833 - dense_1_acc_4: 0.9833 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_ac

60/60 [==============================] - 1s 12ms/step - loss: 6.6100 - dense_1_loss_1: 3.7887 - dense_1_loss_2: 1.3764 - dense_1_loss_3: 0.4494 - dense_1_loss_4: 0.1204 - dense_1_loss_5: 0.0750 - dense_1_loss_6: 0.0563 - dense_1_loss_7: 0.0475 - dense_1_loss_8: 0.0374 - dense_1_loss_9: 0.0366 - dense_1_loss_10: 0.0311 - dense_1_loss_11: 0.0361 - dense_1_loss_12: 0.0323 - dense_1_loss_13: 0.0284 - dense_1_loss_14: 0.0288 - dense_1_loss_15: 0.0289 - dense_1_loss_16: 0.0290 - dense_1_loss_17: 0.0288 - dense_1_loss_18: 0.0308 - dense_1_loss_19: 0.0303 - dense_1_loss_20: 0.0315 - dense_1_loss_21: 0.0314 - dense_1_loss_22: 0.0295 - dense_1_loss_23: 0.0297 - dense_1_loss_24: 0.0288 - dense_1_loss_25: 0.0303 - dense_1_loss_26: 0.0309 - dense_1_loss_27: 0.0355 - dense_1_loss_28: 0.0337 - dense_1_loss_29: 0.0365 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.6333 - dense_1_acc_3: 0.8833 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_ac

60/60 [==============================] - 1s 12ms/step - loss: 6.3656 - dense_1_loss_1: 3.7727 - dense_1_loss_2: 1.3106 - dense_1_loss_3: 0.4049 - dense_1_loss_4: 0.1063 - dense_1_loss_5: 0.0667 - dense_1_loss_6: 0.0496 - dense_1_loss_7: 0.0419 - dense_1_loss_8: 0.0332 - dense_1_loss_9: 0.0322 - dense_1_loss_10: 0.0274 - dense_1_loss_11: 0.0317 - dense_1_loss_12: 0.0283 - dense_1_loss_13: 0.0250 - dense_1_loss_14: 0.0253 - dense_1_loss_15: 0.0254 - dense_1_loss_16: 0.0257 - dense_1_loss_17: 0.0252 - dense_1_loss_18: 0.0272 - dense_1_loss_19: 0.0268 - dense_1_loss_20: 0.0275 - dense_1_loss_21: 0.0277 - dense_1_loss_22: 0.0258 - dense_1_loss_23: 0.0261 - dense_1_loss_24: 0.0254 - dense_1_loss_25: 0.0266 - dense_1_loss_26: 0.0273 - dense_1_loss_27: 0.0313 - dense_1_loss_28: 0.0296 - dense_1_loss_29: 0.0320 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.6333 - dense_1_acc_3: 0.8833 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_ac

60/60 [==============================] - 1s 12ms/step - loss: 6.1586 - dense_1_loss_1: 3.7568 - dense_1_loss_2: 1.2525 - dense_1_loss_3: 0.3665 - dense_1_loss_4: 0.0950 - dense_1_loss_5: 0.0597 - dense_1_loss_6: 0.0441 - dense_1_loss_7: 0.0376 - dense_1_loss_8: 0.0298 - dense_1_loss_9: 0.0287 - dense_1_loss_10: 0.0245 - dense_1_loss_11: 0.0283 - dense_1_loss_12: 0.0252 - dense_1_loss_13: 0.0222 - dense_1_loss_14: 0.0226 - dense_1_loss_15: 0.0226 - dense_1_loss_16: 0.0229 - dense_1_loss_17: 0.0225 - dense_1_loss_18: 0.0242 - dense_1_loss_19: 0.0239 - dense_1_loss_20: 0.0245 - dense_1_loss_21: 0.0246 - dense_1_loss_22: 0.0230 - dense_1_loss_23: 0.0232 - dense_1_loss_24: 0.0227 - dense_1_loss_25: 0.0237 - dense_1_loss_26: 0.0244 - dense_1_loss_27: 0.0279 - dense_1_loss_28: 0.0265 - dense_1_loss_29: 0.0285 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.6500 - dense_1_acc_3: 0.9000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_ac

60/60 [==============================] - 1s 14ms/step - loss: 5.9790 - dense_1_loss_1: 3.7414 - dense_1_loss_2: 1.1993 - dense_1_loss_3: 0.3320 - dense_1_loss_4: 0.0862 - dense_1_loss_5: 0.0541 - dense_1_loss_6: 0.0396 - dense_1_loss_7: 0.0340 - dense_1_loss_8: 0.0271 - dense_1_loss_9: 0.0259 - dense_1_loss_10: 0.0220 - dense_1_loss_11: 0.0255 - dense_1_loss_12: 0.0227 - dense_1_loss_13: 0.0199 - dense_1_loss_14: 0.0205 - dense_1_loss_15: 0.0204 - dense_1_loss_16: 0.0206 - dense_1_loss_17: 0.0203 - dense_1_loss_18: 0.0218 - dense_1_loss_19: 0.0215 - dense_1_loss_20: 0.0221 - dense_1_loss_21: 0.0222 - dense_1_loss_22: 0.0207 - dense_1_loss_23: 0.0207 - dense_1_loss_24: 0.0204 - dense_1_loss_25: 0.0213 - dense_1_loss_26: 0.0221 - dense_1_loss_27: 0.0251 - dense_1_loss_28: 0.0239 - dense_1_loss_29: 0.0256 - dense_1_loss_30: 0.0000e+00 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.6833 - dense_1_acc_3: 0.9333 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_ac

In [27]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100)
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
Generated 101 sounds using the predicted values for the set of chords ("1") and after pruning
Generated 101 sounds using the predicted values for the set of chords ("2") and after pruning
Generated 101 sounds using the predicted values for the set of chords ("3") and after pruning
Generated 101 sounds using the predicted values for the set of chords ("4") and after pruning
Generated 100 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi


In [28]:
play_music('output/my_music.midi')